# The Grammar of Data Wrangling

Let's go over some of the most used wrangling functions:

**Topic:** `filter()`, `select()`, `mutate()`, `group_by()`, `summarize()`, `arrange()`

You can find much more in the documentation for [wrangling](https://dplyr.tidyverse.org/articles/dplyr.html), `dplyr` [cheatsheet](https://rstudio.github.io/cheatsheets/data-transformation.pdf).


In [ ]:
library(dplyr)
library(palmerpenguins)   # provides the penguins dataset
library(tibble)

# Warning messages come from overwritting (masking) variables declared in other packages. 
# So penguins is from the package dataset which is already loaded.
# Then we load palmerpenguins which has penguins, too. 

# Use this to surpress warning and prevent them from showing
# when you know you're doing the correct thing.

# suppressPackageStartupMessages({
#   library(dplyr)
#   library(palmerpenguins)   # provides penguins and penguins_raw
#   library(tibble)
# })

penguins |> glimpse()

# `filter()`

`filter()` keeps **rows** that match conditions.


In [ ]:
# Keep only Adelie penguins
penguins |> 
  filter(species == "Adelie") |>
  glimpse()

Try it:
- Change `"Adelie"` to `"Chinstrap"` or `"Gentoo"`.
- Filter for a specific island (e.g., `"Biscoe"`).
- Filter out missing values with `!is.na(...)`.


In [ ]:
# Your work area

In [ ]:
# Examples: multiple filter conditions
penguins |>
  filter(
    island == "Biscoe",
    !is.na(sex),
    bill_length_mm >= 40
  ) |>
  glimpse()

## Review: `=` versus `==`

- `=` and `<-` **assign** values (create/overwrite variables).
- `==` **tests** whether two values are equal.

So `filter(species == "Adelie")` keeps only rows where the *species* value equals `"Adelie"`.


## `>=` and `<=`

- `>=` means “greater than or equal to”
- `<=` means “less than or equal to”

Example: `filter(body_mass_g >= 4000)` keeps penguins with body mass at least 4000g.


In [ ]:
penguins |>
  filter(body_mass_g >= 4000) |>
  glimpse()

# `select()`

`select()` keeps (or reorders) **columns**. You can also rename columns inside `select()`.


In [ ]:
penguins |>
  select(
    sex,
    bill_len = bill_length_mm,
    bill_depth = bill_depth_mm,
    flipper = flipper_length_mm,
    mass_g = body_mass_g
  ) |>
  glimpse()

Try it:
- Add or remove columns.
- Rename a column by using `new_name = old_name`.


# `mutate()`

`mutate()` creates **new columns** 

In [ ]:
penguins |>
  mutate(
    body_mass_kg = body_mass_g / 1000,
    flipper_length_cm = flipper_length_mm / 10
  ) |>
  select(species, island, sex, body_mass_g, body_mass_kg, flipper_length_mm, flipper_length_cm) |>
  glimpse()

or overwrites existing columns. 

`penguins` is pretty clean. A good reason to overwrite a column might be to change to a more appropriate data type. 

Let's check out some messier data:

In [ ]:
penguins_raw |>
    glimpse()

What types do you think we should use?

In [ ]:
penguins_raw |>
  mutate(
    # overwrite the existing column with a cleaned version
    `Body Mass (g)` = as.integer(`Body Mass (g)`)
  ) |>
  glimpse()

We'll come back to this later in the notebook.

Try it:
- Create a new variable that is **three times** the value of `bill_depth_mm`.
- Create `bill_length_sq = bill_length_mm^2`.

In [ ]:
# Your work area

# `group_by()` + `summarize()` + `arrange()`

A very common pattern:
1. group by a category,
2. compute summary statistics per group,
3. sort (rank) the results.

In [ ]:
names(penguins)

In [ ]:
penguins |>
  group_by(species) |>
  summarize(
    mean_mass_g = mean(body_mass_g),
    median_flipper_mm = median(flipper_length_mm),
    max_bill_ratio = max(bill_depth_mm),
  ) |>
  arrange(desc(mean_mass_g))

Filter out the missing values (or NAs) or set `na.rm` to TRUE to remove them. 

In [ ]:
penguins |>
  group_by(species) |>
  summarize(
    mean_mass_g = mean(body_mass_g, na.rm = TRUE),
    median_flipper_mm = median(flipper_length_mm, na.rm = TRUE),
    max_bill_ratio = max(bill_depth_mm, na.rm = TRUE),
  ) |>
  arrange(desc(mean_mass_g))

## Using `across()` to apply the same function to multiple columns

This is handy when you want “the mean of several numeric columns” (or median, sd, etc.).


In [ ]:
names(penguins)

In [ ]:
penguins |>
  group_by(species) |>
  summarize(
    across(
      c(body_mass_g, flipper_length_mm, bill_depth_mm),
      \(x) mean(x, na.rm = TRUE),
      .names = "mean_{col}")
    ) |>
  arrange(desc(mean_body_mass_g))

# Practice

1. Group by **island** (instead of species) and compute:
   - `mean()` body mass,
   - `median()` flipper length,
   - and `count()` of observations.

2. Filter to `island == "Biscoe"` and group by **sex**:
   - What is the mean mass for each sex?

3. Use `across()` to compute the mean of `bill_len`, `bill_depth`, `flipper_mm`, and `mass_g` for each **(species, island)** pair.


In [ ]:
# Your work area


# Cleaning `penguins_raw` Example

What is wrong with this data? What makes it messy?


## What makes data *clean* (practical checklist)

Clean data is **trustworthy** and **consistent**, so analysis works the way you expect.

- **Clear variable names** (consistent style, descriptive, no hidden units)
- **Correct data types** (numbers as numeric, dates as dates, categories as factors/characters)
- **Consistent coding of categories** (no `"male "` vs `"Male"` vs `"M"`)
- **Consistent units** (no mix of mm/cm or g/kg in the same column)
- **Missing values encoded consistently** (`NA`, not `""`, `"N/A"`, `-999`)
- **Valid values** (ranges make sense; no impossible measurements)
- **Duplicates handled** (no unintended repeated rows)
- **Documented meaning** (you can explain what each column is and the units/coding)

## Step 1: Take a look at the raw data

In [ ]:
penguins_raw |> glimpse()

## Step 2: Make column names consistent

`penguins_raw` has long names and punctuation. A common first step is to standardize names to **snake_case**.


In [ ]:
library(janitor) # to clean_names

raw1 <- penguins_raw |> 
  clean_names() # Removes spacing and cases from variables names to make them easier to work with. Now we don't have to use ``.

raw1 |> glimpse()

## Step 3: Keep the columns you actually want

`penguins_raw` includes some columns used for labeling/notes. Start by selecting the core variables.


In [ ]:
# Only include what your want
raw2 <- raw1 |>
  select(
    species,
    region,
    island,
    stage,
    clutch_completion,
    date_egg,
    sex,
    culmen_length_mm,
    culmen_depth_mm,
    flipper_length_mm,
    body_mass_g
  )

raw2 |> glimpse()

In [ ]:
# Another way is to just say don't include what your don't want

raw2 <- raw1 |>
  select(
    -c(comments,
       study_name,
       sample_number,
       individual_id,
       delta_15_n_o_oo,
       delta_13_c_o_oo)
  )

raw2 |> glimpse()

## Step 4: Decide how to handle missingness

Two common approaches:
- **Drop rows** missing key variables (simple for teaching)
- **Keep missing rows** but be explicit (`na.rm = TRUE` in summaries)

Here we drop rows missing core measurements we might like to measure or visualize.


In [ ]:
raw3 <- raw2 |>
  filter(
    !is.na(species),
    !is.na(island),
    !is.na(culmen_length_mm),
    !is.na(culmen_depth_mm),
    !is.na(flipper_length_mm),
    !is.na(body_mass_g)
  )

raw3 |> glimpse()


## Step 5: change values to be consistent


Common issues:
- extra spaces (leading/trailing)
- inconsistent capitalization

We’ll make several the sex column lower case


In [ ]:
raw4 <- raw3 |>
  mutate(sex = tolower(sex)) #sex is lower case in penguin data

raw4 |> glimpse()

This step is a bit subjective. You could do other things with this data set like shorten the species names. 

## Step 6: Convert everything to their appropriate types

In [ ]:
raw5 <- raw4 |>
  mutate(
    # integers (IDs / counts)
    body_mass_g = as.integer(body_mass_g),
    flipper_length_mm = as.integer(flipper_length_mm),

    # dates
    date_egg = as.Date(date_egg),  # should already work if it's a Date-like string

    # categorical variables
    species = factor(species),
    region  = factor(region),
    island  = factor(island),
    stage   = factor(stage),
    sex = factor(sex),

    # logical
    clutch_completion = ifelse(clutch_completion == "Yes", TRUE, FALSE),
  )

raw5 |> glimpse()

## Step 7: Add variables you would like

- Create derived variables with clear units


In [ ]:
penguins_raw_clean <- raw5 |>
  mutate(
    body_mass_kg = body_mass_g / 1000,
    bill_ratio = culmen_length_mm / culmen_depth_mm
  ) 

penguins_raw_clean |> glimpse()